In [8]:
!pip install transformers pandas peft torch datasets tensorboard

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


### lora를 이용한 Qwen/Qwen2.5-7B-Instruct 모델 파인튜닝

In [9]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
import torch

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# CSV 파일 로드
df = pd.read_csv("./data/merged_museum_data.csv").head(1000)

# 파인튜닝 형식 (질문과 답변형식)으로 데이터 전처리
def format_data(row):
    title = row['Title']
    description = row['Description']
    image_url = row['Image URL']
    
    return [
        {
            "prompt": f"이 유물의 이름이 뭐야? ({title})",
            "response": title
        },
        {
            "prompt": f"어떤 유물이야? ({title})",
            "response": description
        },
        {
            "prompt": f"유물의 특징을 설명해줘. ({title})",
            "response": description
        },
        {
            "prompt": f"유물의 사진을 보여줄 수 있어? ({title})",
            "response": image_url
        },
        {
            "prompt": f"이 유물은 어느 시대의 유물인가요? ({title})",
            "response": "정보 없음"  # 시대 정보가 없을 경우 "정보 없음"
        }
    ]

# 각 유물에 대해 질문과 답변을 생성하고 리스트로 변환
formatted_data = df.apply(format_data, axis=1).explode().tolist()

# Dataset 객체로 변환
dataset = Dataset.from_list(formatted_data)

# 토크나이저 및 모델 로드
model_name = "Qwen/Qwen2.5-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)

# tokenizer의 pad_token을 eos_token으로 설정
tokenizer.pad_token = tokenizer.eos_token

# 모델과 토크나이저 확인
# print(f"Model: {model_name} loaded.")
# print(f"Tokenizer: {tokenizer}")

# # 데이터셋 예시 출력 (최대 5개)
# print(dataset[:5])



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:


lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)


model = get_peft_model(model, lora_config)

In [12]:
from peft import get_peft_model_state_dict

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params}")

Trainable parameters: 40370176


In [13]:
def tokenize_function(examples):
    # 'prompt'와 'response'를 사용하여 입력 문자열 생성
    inputs = [f"<|im_start|>user\n{instr}<|im_end|>\n<|im_start|>assistant\n{resp}<|im_end|>"
              for instr, resp in zip(examples["prompt"], examples["response"])]
    
    # 토크나이징
    tokenized = tokenizer(inputs, padding="max_length", truncation=True, max_length=1024)

    # 'labels' 추가 (패딩 토큰에 -100 설정)
    tokenized["labels"] = [
        [-100 if token_id == tokenizer.pad_token_id else token_id for token_id in ids]
        for ids in tokenized["input_ids"]
    ]

    return tokenized

# 데이터셋에서 'prompt'와 'response' 컬럼을 제거하고, 토크나이징 처리
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["prompt", "response"])

# 학습과 테스트 데이터셋 분리 (80% 학습, 20% 테스트)
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)

# 학습과 테스트 데이터셋 출력 확인
print(train_test_split)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})


In [ ]:
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):  # Use **kwargs to capture any extra arguments
        outputs = model(**inputs)
        loss = outputs.loss
        print(f"Loss: {loss.item()}", flush=True)  # Print loss immediately
        return (loss, outputs) if return_outputs else loss


# 학습 설정
training_args = TrainingArguments(
    output_dir="./qwen2.5-7b-data_final_finetuned",
    num_train_epochs=5,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,  # 학습률을 5e-5로 변경
    fp16=False,  
    bf16=True,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    logging_dir="./logs",
    report_to="tensorboard",
    save_total_limit=1,
    max_grad_norm=1.0    # Gradient Clipping 추가 (옵션)
)

# 데이터셋 준비 (생략된 부분은 그대로)
# model과 tokenizer 로드
model_name = "Qwen/Qwen2.5-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# LoraConfig 설정
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

# trainer에서 label_names 제거
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_test_split["train"],
    eval_dataset=train_test_split["test"],
    tokenizer=tokenizer
)

# 학습 및 저장
trainer.train()
model.save_pretrained("./qwen2.5-7b-finetuned/final_model")
tokenizer.save_pretrained("./qwen2.5-7b-finetuned/final_model")





Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]